<a href="https://colab.research.google.com/github/SruthiM-10/Irrigation-Scheduling-Recommendations-And-Dataset/blob/main/(4)_filling_null_values.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from openai import OpenAI
import re
import json
import requests
import os

In [ ]:
! pip install -q langchain_community langchain_openai faiss-cpu langchain langchain_core langgraph pinecone PyPDF2 doi2pdf

In [ ]:
import time
import uuid
import requests
from langchain_community.document_loaders import WebBaseLoader
from typing import Dict, List, Any, Tuple, Optional, Union
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage
import pickle
from langchain_core.pydantic_v1 import Field, conlist, BaseModel

import warnings

warnings.filterwarnings(
    "ignore",
    category=UserWarning,
    module='langchain_openai.chat_models.base'
)

/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
all_samples = pd.read_csv("/content/drive/MyDrive/1:1_Sruthi_Manoj/Dataset/csv_files/Extra analysis/cleaned_samples (6).csv")
all_samples.drop("Unnamed: 0", axis= 1, inplace= True)
all_samples

,irrigation_scheduling_method,irrigation_scheduling_method_standardized,plant_yield,plant_yield_kg_per_hectare,plant_yield_notes,location,latitude,latitude_decimal_degrees,latitude_notes,lat_filled,...,water_productivity_kg_per_m^3,water_productivity_notes,water_productivity_valid,water_productivity_filled,file_path,doi,doi_url,processed_at,content source,relevance_score
0,Soil moisture sensor-based method (T-VWC),Soil Moisture-Based,66.77 t ha−1,60572.74245,This value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,40.178600,Original value was in degrees with a direction...,False,...,17.370000,NaN,NaN,False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
1,Accumulated radiation combined with soil moist...,Soil Moisture-Based,58.98 t ha−1,53505.77130,Yield value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,40.178600,Original value was in degrees with a direction...,False,...,20.520000,NaN,NaN,False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
2,Crop evapotranspiration estimated method using...,Evapotranspiration-Based,64.54 t ha−1,58549.71990,Value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,40.178600,Original value was in degrees with a direction...,False,...,25.070000,NaN,NaN,False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
3,Soil water balance method,Soil Moisture-Based,"5,085 kg ha−1 (100% ETc), 4,830 kg ha−1 (85% E...",5085.00000,100% ETc,"Melkassa Agricultural Research Center, Awash B...",8°24′N,8.400000,Converted from degrees and minutes format (8°2...,False,...,13.583452,(70% ETc),False,True,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1177/11786221241274019,https://doi.org/10.1177/11786221241274019,2025-09-12 04:01:02,pdf,9
4,Regulated Deficit Irrigation,Deficit/Partial Irrigation,2.406 kg m-2,24060.00000,"Value is in kilograms per square meter, which ...","Maringá, Paraná, Brazil",23º25′S,-23.416667,Converted from degrees and minutes format (23º...,False,...,13.583452,NaN,NaN,True,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1093/jxb/erv034,https://doi.org/10.1093/jxb/erv034,2025-09-12 04:06:41,pdf,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309,"100% ETc, 4-day interval",Evapotranspiration-Based,"136.78 t/ha (Spring 2023), 50.10 t/ha (Autumn ...",45449.96850,Yield value for Autumn 2023.,"Yangling, Shaanxi, China",34°20′N,34.333333,Original value was in degrees and minutes form...,False,...,19.300000,Autumn 2023,True,False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3390/agronomy15051068,https://doi.org/10.3390/agronomy15051068,2025-09-12 17:23:02,pdf,10
310,Timing of last irrigation,Conventional/Fixed Scheduling,1669-2405 kg/ha,2405.00000,Value is already in kg/ha.,"Gezira Research Station Farm, Wad Medani, Sudan",NaN,14.401180,NaN,True,...,0.710000,Maximum value of the range 0.45-0.71 kg/m3,True,False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.21608/sjdr.2020.366016074,https://doi.org/10.21608/sjdr.2020.366016074,2025-09-12 23:58:14,pdf,8
311,Film-Mulched Drip Irrigation,Conventional/Fixed Scheduling,"6218.07 kg ha−1 (2018), 5660.59 kg ha−1 (2019)",5660.59000,Yield value for the year 2019.,"Korla, Xinjiang, China",40°53′N,40.883333,Original value was in degrees and minutes form...,False,...,2.600000,(W 0.8),True,False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3390/agronomy15071674,https://doi.org/10.3390/agronomy15071674,2025-09-13 00:18:02,pdf,9
312,Film-Mulched Drip I

Here's the state of the dataset before filling --

In [ ]:
# only complete pdfs files
info = []
for col in ["location", "latitude_decimal_degrees", "longitude_decimal_degrees", "altitude", "plant_type_standardized", "plant_yield_kg_per_hectare", "water_productivity_kg_per_m^3", "irrigation_scheduling_method"]:
  valid = all_samples.loc[(all_samples[col] != "NA") & (all_samples[col].notna()) & (all_samples["content source"] == "pdf")]
  total_pdf = all_samples.loc[all_samples["content source"] == "pdf"]
  info.append({
      'feature': col,
      'total_samples': len(total_pdf),
      'valid_samples': len(valid),
      'percentage': float(len(valid)/len(total_pdf)) * 100.0
  })
info = pd.DataFrame(info)
info.sort_values(by= "percentage", ascending= False, inplace= True)
info

,feature,total_samples,valid_samples,percentage
0,location,316,316,100.000000
7,irrigation_scheduling_method,316,316,100.000000
5,plant_yield_kg_per_hectare,316,316,100.000000
4,plant_type_standardized,316,316,100.000000
1,latitude_decimal_degrees,316,221,69.936709
2,longitude_decimal_degrees,316,221,69.936709
3,altitude,316,186,58.860759
6,water_productivity_kg_per_m^3,316,185,58.544304


This is to track which values were originally null and then filled.

In [ ]:
all_samples["lat_filled"] = all_samples["latitude"].isna()
all_samples["long_filled"] = all_samples["longitude"].isna()
# converted to decimal degrees :)
all_samples["altitude_filled"] = all_samples["altitude"].isna() # all altitude values are already in meters
all_samples["water_productivity_filled"] = all_samples["water_productivity_kg_per_m^3"].isna()

In [ ]:
all_samples.loc[0, 'location'] # test

'Xiaotangshan National Precision Agriculture Experiment Station, Beijing, China'

# Latitude/Longitude

In [ ]:
api_key = "4353f25480414794ae44c1c458d81258" # open cage (forward geocoding)
base_url = f"https://api.opencagedata.com/geocode/v1/json?key={api_key}"
response = requests.get(f"{base_url}&q={all_samples.loc[0, 'location']}")
response.json()

{'documentation': 'https://opencagedata.com/api',
 'licenses': [{'name': 'see attribution guide',
   'url': 'https://opencagedata.com/credits'}],
 'rate': {'limit': 2500, 'remaining': 2499, 'reset': 1759622400},
 'results': [{'annotations': {'DMS': {'lat': "39° 54' 27.00000'' N",
     'lng': "116° 23' 50.02800'' E"},
    'MGRS': '50SMK4847817664',
    'Maidenhead': 'OM89ev77qt',
    'Mercator': {'x': 12957280.373, 'y': 4825091.628},
    'OSM': {'note_url': 'https://www.openstreetmap.org/note/new#map=17/39.90750/116.39723&layers=N',
     'url': 'https://www.openstreetmap.org/?mlat=39.90750&mlon=116.39723#map=17/39.90750/116.39723'},
    'UN_M49': {'regions': {'ASIA': '142',
      'CN': '156',
      'EASTERN_ASIA': '030',
      'WORLD': '001'},
     'statistical_groupings': ['LEDC']},
    'callingcode': 86,
    'currency': {'alternate_symbols': ['CN¥', '元', 'CN元'],
     'decimal_mark': '.',
     'html_entity': '￥',
     'iso_code': 'CNY',
     'iso_numeric': '156',
     'name': 'Chinese 

In [ ]:
response = response.json()
response.get("results")[0].get("geometry")

{'lat': 39.9075, 'lng': 116.39723}

In [ ]:
def fill_lat_lon(row):
  lat = row['latitude_decimal_degrees']
  lon = row['longitude_decimal_degrees']
  if not pd.isna(lat) and not pd.isna(lon):
    return row

  location = row['location']

  response = requests.get(f"{base_url}&q={location}")
  if response.status_code != 200:
    print(response.status_code)
    return row

  print("Location: {}".format(location))
  if pd.isna(lat):
    try:
      row['latitude_decimal_degrees'] = response.json().get("results")[0].get("geometry").get("lat")
      print("Added lat - {}".format(row['latitude_decimal_degrees']))
    except Exception as e:
      print(response.json())
  if pd.isna(lon):
    try:
      row['longitude_decimal_degrees'] = response.json().get("results")[0].get("geometry").get("lng")
      print('Added lng - {}'.format(row['longitude_decimal_degrees']))
    except Exception as e:
      print(response.json())

  return row

df = all_samples.apply(fill_lat_lon, axis= 1)
df

Location: China
Added lat - 35.000074
Added lng - 104.999927
Location: China
Added lat - 35.000074
Added lng - 104.999927
Location: China
Added lat - 35.000074
Added lng - 104.999927
Location: China
Added lat - 35.000074
Added lng - 104.999927
Location: China
Added lat - 35.000074
Added lng - 104.999927
Location: China
Added lat - 35.000074
Added lng - 104.999927
Location: China
Added lat - 35.000074
Added lng - 104.999927
Location: China
Added lat - 35.000074
Added lng - 104.999927
Location: China
Added lat - 35.000074
Added lng - 104.999927
Location: China
Added lat - 35.000074
Added lng - 104.999927
Location: China
Added lat - 35.000074
Added lng - 104.999927
Location: China
Added lat - 35.000074
Added lng - 104.999927
Location: China
Added lat - 35.000074
Added lng - 104.999927
Location: China
Added lat - 35.000074
Added lng - 104.999927
Location: China
Added lat - 35.000074
Added lng - 104.999927
Location: China
Added lat - 35.000074
Added lng - 104.999927
Location: China
Added la

,irrigation_scheduling_method,plant_yield,plant_yield_kg_per_hectare,plant_yield_notes,location,latitude,latitude_decimal_degrees,latitude_notes,longitude,longitude_decimal_degrees,...,file_path,doi,doi_url,processed_at,content source,relevance_score,lat_filled,long_filled,altitude_filled,water_productivity_filled
0,Soil moisture sensor-based method (T-VWC),66.77 t ha−1,60572.74245,This value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,40.178600,Original value was in degrees with a direction...,116.4442° E,116.444200,...,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9,False,False,True,False
1,Accumulated radiation combined with soil moist...,58.98 t ha−1,53505.77130,Yield value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,40.178600,Original value was in degrees with a direction...,116.4442° E,116.444200,...,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9,False,False,True,False
2,Crop evapotranspiration estimated method using...,64.54 t ha−1,58549.71990,Value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,40.178600,Original value was in degrees with a direction...,116.4442° E,116.444200,...,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9,False,False,True,False
3,Soil water balance method,"5,085 kg ha−1 (100% ETc), 4,830 kg ha−1 (85% E...",5085.00000,100% ETc,"Melkassa Agricultural Research Center, Awash B...",8°24′N,8.400000,Converted from degrees and minutes format (8°2...,39°21′E,39.350000,...,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1177/11786221241274019,https://doi.org/10.1177/11786221241274019,2025-09-12 04:01:02,pdf,9,False,False,False,True
4,Regulated Deficit Irrigation,2.406 kg m-2,24060.00000,"Value is in kilograms per square meter, which ...","Maringá, Paraná, Brazil",23º25′S,-23.416667,Converted from degrees and minutes format (23º...,51º57′W,-51.950000,...,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1093/jxb/erv034,https://doi.org/10.1093/jxb/erv034,2025-09-12 04:06:41,pdf,9,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,"100% ETc, 4-day interval","136.78 t/ha (Spring 2023), 50.10 t/ha (Autumn ...",45449.96850,Yield value for Autumn 2023.,"Yangling, Shaanxi, China",34°20′N,34.333333,Original value was in degrees and minutes form...,108°04′E,108.066667,...,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3390/agronomy15051068,https://doi.org/10.3390/agronomy15051068,2025-09-12 17:23:02,pdf,10,False,False,False,False
312,Timing of last irrigation,1669-2405 kg/ha,2405.00000,Value is already in kg/ha.,"Gezira Research Station Farm, Wad Medani, Sudan",NaN,14.401180,NaN,NaN,33.519890,...,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.21608/sjdr.2020.366016074,https://doi.org/10.21608/sjdr.2020.366016074,2025-09-12 23:58:14,pdf,8,True,True,True,False
313,Film-Mulched Drip Irrigation,"6218.07 kg ha−1 (2018), 5660.59 kg ha−1 (2019)",5660.59000,Yield value for the year 2019.,"Korla, Xinjiang, China",40°53′N,40.883333,Original value was in degrees and minutes form...,86°56′E,86.933333,...,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3390/agronomy15071674,https://doi.org/10.3390/agronomy15071674,2025-09-13 00:18:02,pdf,9,False,False,False,False
314,Film-Mulched Drip Irrigation,"6218.07 kg ha−1 (2018), 5660.59 kg ha−1 (2019)",5660.59000,Yield value for the year 2019.,"Korla, Xinjiang, China",40°53′N,40.883333,Original value was in degrees and minutes form...,86°56′E,86.93333

In [ ]:
df.dropna(subset=["latitude_decimal_degrees", "longitude_decimal_degrees"], axis=0, inplace=True)

In [ ]:
# only complete pdfs files
info = []
for col in ["location", "latitude_decimal_degrees", "longitude_decimal_degrees", "altitude", "plant_type_standardized", "plant_yield_kg_per_hectare", "water_productivity_kg_per_m^3", "irrigation_scheduling_method"]:
  valid = df.loc[(df[col] != "NA") & (df[col].notna()) & (df["content source"] == "pdf")]
  total_pdf = df.loc[df["content source"] == "pdf"]
  info.append({
      'feature': col,
      'total_samples': len(total_pdf),
      'valid_samples': len(valid),
      'percentage': float(len(valid)/len(total_pdf)) * 100.0
  })
info = pd.DataFrame(info)
info.sort_values(by= "percentage", ascending= False, inplace= True)
info

,feature,total_samples,valid_samples,percentage
0,location,314,314,100.000000
1,latitude_decimal_degrees,314,314,100.000000
2,longitude_decimal_degrees,314,314,100.000000
4,plant_type_standardized,314,314,100.000000
7,irrigation_scheduling_method,314,314,100.000000
5,plant_yield_kg_per_hectare,314,314,100.000000
3,altitude,314,186,59.235669
6,water_productivity_kg_per_m^3,314,185,58.917197


# Altitude

In [ ]:
base_url = "https://api.open-meteo.com/v1/elevation?latitude={lat}&longitude={lon}"
response = requests.get(base_url.format(lat=52.52,lon=13.41))
response.json()

{'elevation': [38.0]}

In [ ]:
def fill_altitude(row):
  lat = row['latitude_decimal_degrees']
  lon = row['longitude_decimal_degrees']
  altitude = row['altitude']
  if not pd.isna(altitude):
    return row

  location = row['location']

  response = requests.get(base_url.format(lat=lat,lon=lon))
  if response.status_code != 200:
    print(response.status_code)
    return row

  print("Location: {}".format(location))

  row['altitude'] = response.json().get("elevation")
  print('Added altitude - {}'.format(row['altitude']))

  return row

next_df = df.apply(fill_altitude, axis= 1)
next_df

Location: Xiaotangshan National Precision Agriculture Experiment Station, Beijing, China
Added altitude - [37.0]
Location: Xiaotangshan National Precision Agriculture Experiment Station, Beijing, China
Added altitude - [37.0]
Location: Xiaotangshan National Precision Agriculture Experiment Station, Beijing, China
Added altitude - [37.0]
Location: China
Added altitude - [2001.0]
Location: China
Added altitude - [2001.0]
Location: China
Added altitude - [2001.0]
Location: China
Added altitude - [2001.0]
Location: China
Added altitude - [2001.0]
Location: China
Added altitude - [2001.0]
Location: China
Added altitude - [2001.0]
Location: China
Added altitude - [2001.0]
Location: China
Added altitude - [2001.0]
Location: China
Added altitude - [2001.0]
Location: China
Added altitude - [2001.0]
Location: China
Added altitude - [2001.0]
Location: China
Added altitude - [2001.0]
Location: China
Added altitude - [2001.0]
Location: China
Added altitude - [2001.0]
Location: China
Added altitude 

,irrigation_scheduling_method,plant_yield,plant_yield_kg_per_hectare,plant_yield_notes,location,latitude,latitude_decimal_degrees,latitude_notes,longitude,longitude_decimal_degrees,...,file_path,doi,doi_url,processed_at,content source,relevance_score,lat_filled,long_filled,altitude_filled,water_productivity_filled
0,Soil moisture sensor-based method (T-VWC),66.77 t ha−1,60572.74245,This value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,40.178600,Original value was in degrees with a direction...,116.4442° E,116.444200,...,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9,False,False,True,False
1,Accumulated radiation combined with soil moist...,58.98 t ha−1,53505.77130,Yield value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,40.178600,Original value was in degrees with a direction...,116.4442° E,116.444200,...,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9,False,False,True,False
2,Crop evapotranspiration estimated method using...,64.54 t ha−1,58549.71990,Value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,40.178600,Original value was in degrees with a direction...,116.4442° E,116.444200,...,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9,False,False,True,False
3,Soil water balance method,"5,085 kg ha−1 (100% ETc), 4,830 kg ha−1 (85% E...",5085.00000,100% ETc,"Melkassa Agricultural Research Center, Awash B...",8°24′N,8.400000,Converted from degrees and minutes format (8°2...,39°21′E,39.350000,...,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1177/11786221241274019,https://doi.org/10.1177/11786221241274019,2025-09-12 04:01:02,pdf,9,False,False,False,True
4,Regulated Deficit Irrigation,2.406 kg m-2,24060.00000,"Value is in kilograms per square meter, which ...","Maringá, Paraná, Brazil",23º25′S,-23.416667,Converted from degrees and minutes format (23º...,51º57′W,-51.950000,...,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1093/jxb/erv034,https://doi.org/10.1093/jxb/erv034,2025-09-12 04:06:41,pdf,9,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,"100% ETc, 4-day interval","136.78 t/ha (Spring 2023), 50.10 t/ha (Autumn ...",45449.96850,Yield value for Autumn 2023.,"Yangling, Shaanxi, China",34°20′N,34.333333,Original value was in degrees and minutes form...,108°04′E,108.066667,...,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3390/agronomy15051068,https://doi.org/10.3390/agronomy15051068,2025-09-12 17:23:02,pdf,10,False,False,False,False
312,Timing of last irrigation,1669-2405 kg/ha,2405.00000,Value is already in kg/ha.,"Gezira Research Station Farm, Wad Medani, Sudan",NaN,14.401180,NaN,NaN,33.519890,...,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.21608/sjdr.2020.366016074,https://doi.org/10.21608/sjdr.2020.366016074,2025-09-12 23:58:14,pdf,8,True,True,True,False
313,Film-Mulched Drip Irrigation,"6218.07 kg ha−1 (2018), 5660.59 kg ha−1 (2019)",5660.59000,Yield value for the year 2019.,"Korla, Xinjiang, China",40°53′N,40.883333,Original value was in degrees and minutes form...,86°56′E,86.933333,...,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3390/agronomy15071674,https://doi.org/10.3390/agronomy15071674,2025-09-13 00:18:02,pdf,9,False,False,False,False
314,Film-Mulched Drip Irrigation,"6218.07 kg ha−1 (2018), 5660.59 kg ha−1 (2019)",5660.59000,Yield value for the year 2019.,"Korla, Xinjiang, China",40°53′N,40.883333,Original value was in degrees and minutes form...,86°56′E,86.93333

In [ ]:
class AltitudeItem(BaseModel):
  altitude: float
  cleaning_notes: str

def call_gpt_api(llm_type, prompt: str, max_tokens: int = 1000) -> str:
    """Call GPT API with error handling"""
    try:
        structured_llm = llm.with_structured_output(llm_type)
        system_message_prompt = [SystemMessage(content=prompt)]
        response = structured_llm.invoke(system_message_prompt)
        return response
    except Exception as e:
        print(f"Error calling GPT API: {e}")
        return ""

def standardize_altitude(df: pd.DataFrame) -> pd.DataFrame:
    """Convert altitude to numeric form"""
    print("Standardizing altitude...")

    # Get sample values to understand the format
    sample_values = df['altitude'].dropna().head(10).tolist()
    print(f"Sample altitude values: {sample_values}")

    prompt = """
You are given altitude values.

Task: Extract the number from this altitude value and return:
  1. altitude: the float value
  2. cleaning_notes: any cleaning notes or extra information in original altitude value

Return ONLY as a JSON array of objects.

Altitude value: {value}
    """

    # Extract numerical values and convert
    def extract_and_convert_altitude(row):
        value = row["altitude"]
        if pd.isna(value):
            print("NA??")
            return row

        # Extract numbers from string
        response = call_gpt_api(llm_type= AltitudeItem, prompt= prompt.format(value=value))
        analysis = response.dict()
        print("Original value: {}".format(value))
        print(analysis)

        row['altitude_numeric'] = analysis['altitude']
        row['altitude_notes'] = analysis['cleaning_notes']

        return row

    df = df.apply(extract_and_convert_altitude, axis= 1)

    return df

last_df = standardize_altitude(next_df)
last_df

Standardizing altitude...
Sample altitude values: [[37.0], [37.0], [37.0], '1550 m', '542 m', '542 m', '542 m', '542 m', '1550 m', [2001.0]]
Original value: [37.0]
{'altitude': 37.0, 'cleaning_notes': ''}
Original value: [37.0]
{'altitude': 37.0, 'cleaning_notes': ''}
Original value: [37.0]
{'altitude': 37.0, 'cleaning_notes': ''}
Original value: 1550 m
{'altitude': 1550.0, 'cleaning_notes': 'Altitude value in meters'}
Original value: 542 m
{'altitude': 542.0, 'cleaning_notes': 'm'}
Original value: 542 m
{'altitude': 542.0, 'cleaning_notes': 'Altitude value in meters'}
Original value: 542 m
{'altitude': 542.0, 'cleaning_notes': 'original altitude value in meters'}
Original value: 542 m
{'altitude': 542.0, 'cleaning_notes': 'm'}
Original value: 1550 m
{'altitude': 1550.0, 'cleaning_notes': 'm'}
Original value: [2001.0]
{'altitude': 2001.0, 'cleaning_notes': ''}
Original value: [2001.0]
{'altitude': 2001.0, 'cleaning_notes': ''}
Original value: [2001.0]
{'altitude': 2001.0, 'cleaning_not

,irrigation_scheduling_method,plant_yield,plant_yield_kg_per_hectare,plant_yield_notes,location,latitude,latitude_decimal_degrees,latitude_notes,longitude,longitude_decimal_degrees,...,doi_url,processed_at,content source,relevance_score,lat_filled,long_filled,altitude_filled,water_productivity_filled,altitude_numeric,altitude_notes
0,Soil moisture sensor-based method (T-VWC),66.77 t ha−1,60572.74245,This value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,40.178600,Original value was in degrees with a direction...,116.4442° E,116.444200,...,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9,False,False,True,False,37.0,
1,Accumulated radiation combined with soil moist...,58.98 t ha−1,53505.77130,Yield value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,40.178600,Original value was in degrees with a direction...,116.4442° E,116.444200,...,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9,False,False,True,False,37.0,
2,Crop evapotranspiration estimated method using...,64.54 t ha−1,58549.71990,Value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,40.178600,Original value was in degrees with a direction...,116.4442° E,116.444200,...,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9,False,False,True,False,37.0,
3,Soil water balance method,"5,085 kg ha−1 (100% ETc), 4,830 kg ha−1 (85% E...",5085.00000,100% ETc,"Melkassa Agricultural Research Center, Awash B...",8°24′N,8.400000,Converted from degrees and minutes format (8°2...,39°21′E,39.350000,...,https://doi.org/10.1177/11786221241274019,2025-09-12 04:01:02,pdf,9,False,False,False,True,1550.0,Altitude value in meters
4,Regulated Deficit Irrigation,2.406 kg m-2,24060.00000,"Value is in kilograms per square meter, which ...","Maringá, Paraná, Brazil",23º25′S,-23.416667,Converted from degrees and minutes format (23º...,51º57′W,-51.950000,...,https://doi.org/10.1093/jxb/erv034,2025-09-12 04:06:41,pdf,9,False,False,False,True,542.0,m
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,"100% ETc, 4-day interval","136.78 t/ha (Spring 2023), 50.10 t/ha (Autumn ...",45449.96850,Yield value for Autumn 2023.,"Yangling, Shaanxi, China",34°20′N,34.333333,Original value was in degrees and minutes form...,108°04′E,108.066667,...,https://doi.org/10.3390/agronomy15051068,2025-09-12 17:23:02,pdf,10,False,False,False,False,521.0,original altitude value in meters
312,Timing of last irrigation,1669-2405 kg/ha,2405.00000,Value is already in kg/ha.,"Gezira Research Station Farm, Wad Medani, Sudan",NaN,14.401180,NaN,NaN,33.519890,...,https://doi.org/10.21608/sjdr.2020.366016074,2025-09-12 23:58:14,pdf,8,True,True,True,False,409.0,
313,Film-Mulched Drip Irrigation,"6218.07 kg ha−1 (2018), 5660.59 kg ha−1 (2019)",5660.59000,Yield value for the year 2019.,"Korla, Xinjiang, China",40°53′N,40.883333,Original value was in degrees and minutes form...,86°56′E,86.933333,...,https://doi.org/10.3390/agronomy15071674,2025-09-13 00:18:02,pdf,9,False,False,False,False,900.0,900 m
314,Film-Mulched Drip Irrigation,"6218.07 kg ha−1 (2018), 5660.59 kg ha−1 (2019)",5660.59000,Yield value for the year 2019.,"Korla, Xinjiang, China",40°53′N,40.883333,Original value was in degrees and minutes form...,86°56′E,86.933333,...,https://doi.org/10.3390/agronomy15071674,2025-09-13 00:18:02,pdf,9,False,False,False,False,900.0,900 m


In [ ]:
# only complete pdfs files
info = []
for col in ["location", "latitude_decimal_degrees", "longitude_decimal_degrees", "altitude_numeric", "plant_type_standardized", "plant_yield_kg_per_hectare", "water_productivity_kg_per_m^3", "irrigation_scheduling_method"]:
  valid = last_df.loc[(last_df[col] != "NA") & (last_df[col].notna()) & (last_df["content source"] == "pdf")]
  total_pdf = last_df.loc[last_df["content source"] == "pdf"]
  info.append({
      'feature': col,
      'total_samples': len(total_pdf),
      'valid_samples': len(valid),
      'percentage': float(len(valid)/len(total_pdf)) * 100.0
  })
info = pd.DataFrame(info)
info.sort_values(by= "percentage", ascending= False, inplace= True)
info

,feature,total_samples,valid_samples,percentage
0,location,314,314,100.000000
1,latitude_decimal_degrees,314,314,100.000000
2,longitude_decimal_degrees,314,314,100.000000
3,altitude_numeric,314,314,100.000000
4,plant_type_standardized,314,314,100.000000
5,plant_yield_kg_per_hectare,314,314,100.000000
7,irrigation_scheduling_method,314,314,100.000000
6,water_productivity_kg_per_m^3,314,185,58.917197


# Water Productivity

In [ ]:
mean = last_df["water_productivity_kg_per_m^3"].mean()
final_df = last_df.copy()
final_df["water_productivity_kg_per_m^3"] = last_df["water_productivity_kg_per_m^3"].apply(
    lambda x: mean if pd.isna(x) else x
)
print("Mean is: {}".format(mean))
final_df

Mean is: 13.583452190676365


,irrigation_scheduling_method,plant_yield,plant_yield_kg_per_hectare,plant_yield_notes,location,latitude,latitude_decimal_degrees,latitude_notes,longitude,longitude_decimal_degrees,...,doi_url,processed_at,content source,relevance_score,lat_filled,long_filled,altitude_filled,water_productivity_filled,altitude_numeric,altitude_notes
0,Soil moisture sensor-based method (T-VWC),66.77 t ha−1,60572.74245,This value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,40.178600,Original value was in degrees with a direction...,116.4442° E,116.444200,...,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9,False,False,True,False,37.0,
1,Accumulated radiation combined with soil moist...,58.98 t ha−1,53505.77130,Yield value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,40.178600,Original value was in degrees with a direction...,116.4442° E,116.444200,...,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9,False,False,True,False,37.0,
2,Crop evapotranspiration estimated method using...,64.54 t ha−1,58549.71990,Value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,40.178600,Original value was in degrees with a direction...,116.4442° E,116.444200,...,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9,False,False,True,False,37.0,
3,Soil water balance method,"5,085 kg ha−1 (100% ETc), 4,830 kg ha−1 (85% E...",5085.00000,100% ETc,"Melkassa Agricultural Research Center, Awash B...",8°24′N,8.400000,Converted from degrees and minutes format (8°2...,39°21′E,39.350000,...,https://doi.org/10.1177/11786221241274019,2025-09-12 04:01:02,pdf,9,False,False,False,True,1550.0,Altitude value in meters
4,Regulated Deficit Irrigation,2.406 kg m-2,24060.00000,"Value is in kilograms per square meter, which ...","Maringá, Paraná, Brazil",23º25′S,-23.416667,Converted from degrees and minutes format (23º...,51º57′W,-51.950000,...,https://doi.org/10.1093/jxb/erv034,2025-09-12 04:06:41,pdf,9,False,False,False,True,542.0,m
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,"100% ETc, 4-day interval","136.78 t/ha (Spring 2023), 50.10 t/ha (Autumn ...",45449.96850,Yield value for Autumn 2023.,"Yangling, Shaanxi, China",34°20′N,34.333333,Original value was in degrees and minutes form...,108°04′E,108.066667,...,https://doi.org/10.3390/agronomy15051068,2025-09-12 17:23:02,pdf,10,False,False,False,False,521.0,original altitude value in meters
312,Timing of last irrigation,1669-2405 kg/ha,2405.00000,Value is already in kg/ha.,"Gezira Research Station Farm, Wad Medani, Sudan",NaN,14.401180,NaN,NaN,33.519890,...,https://doi.org/10.21608/sjdr.2020.366016074,2025-09-12 23:58:14,pdf,8,True,True,True,False,409.0,
313,Film-Mulched Drip Irrigation,"6218.07 kg ha−1 (2018), 5660.59 kg ha−1 (2019)",5660.59000,Yield value for the year 2019.,"Korla, Xinjiang, China",40°53′N,40.883333,Original value was in degrees and minutes form...,86°56′E,86.933333,...,https://doi.org/10.3390/agronomy15071674,2025-09-13 00:18:02,pdf,9,False,False,False,False,900.0,900 m
314,Film-Mulched Drip Irrigation,"6218.07 kg ha−1 (2018), 5660.59 kg ha−1 (2019)",5660.59000,Yield value for the year 2019.,"Korla, Xinjiang, China",40°53′N,40.883333,Original value was in degrees and minutes form...,86°56′E,86.933333,...,https://doi.org/10.3390/agronomy15071674,2025-09-13 00:18:02,pdf,9,False,False,False,False,900.0,900 m


In [ ]:
final_df = all_samples.loc[~(all_samples["water_productivity_valid"] == False)]
final_df

,irrigation_scheduling_method,irrigation_scheduling_method_standardized,plant_yield,plant_yield_kg_per_hectare,plant_yield_notes,location,latitude,latitude_decimal_degrees,latitude_notes,lat_filled,...,water_productivity_kg_per_m^3,water_productivity_notes,water_productivity_valid,water_productivity_filled,file_path,doi,doi_url,processed_at,content source,relevance_score
0,Soil moisture sensor-based method (T-VWC),Soil Moisture-Based,66.77 t ha−1,60572.74245,This value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,40.178600,Original value was in degrees with a direction...,False,...,17.370000,NaN,NaN,False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
1,Accumulated radiation combined with soil moist...,Soil Moisture-Based,58.98 t ha−1,53505.77130,Yield value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,40.178600,Original value was in degrees with a direction...,False,...,20.520000,NaN,NaN,False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
2,Crop evapotranspiration estimated method using...,Evapotranspiration-Based,64.54 t ha−1,58549.71990,Value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,40.178600,Original value was in degrees with a direction...,False,...,25.070000,NaN,NaN,False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
4,Regulated Deficit Irrigation,Deficit/Partial Irrigation,2.406 kg m-2,24060.00000,"Value is in kilograms per square meter, which ...","Maringá, Paraná, Brazil",23º25′S,-23.416667,Converted from degrees and minutes format (23º...,False,...,13.583452,NaN,NaN,True,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1093/jxb/erv034,https://doi.org/10.1093/jxb/erv034,2025-09-12 04:06:41,pdf,9
5,Regulated Deficit Irrigation,Deficit/Partial Irrigation,2.540 kg m-2,25400.00000,"Value is in kg per square meter, which can be ...","Maringá, Paraná, Brazil",23º25′S,-23.416667,Converted from degrees and minutes (23º25′S) t...,False,...,13.583452,NaN,NaN,True,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1093/jxb/erv034,https://doi.org/10.1093/jxb/erv034,2025-09-12 04:06:41,pdf,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309,"100% ETc, 4-day interval",Evapotranspiration-Based,"136.78 t/ha (Spring 2023), 50.10 t/ha (Autumn ...",45449.96850,Yield value for Autumn 2023.,"Yangling, Shaanxi, China",34°20′N,34.333333,Original value was in degrees and minutes form...,False,...,19.300000,Autumn 2023,True,False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3390/agronomy15051068,https://doi.org/10.3390/agronomy15051068,2025-09-12 17:23:02,pdf,10
310,Timing of last irrigation,Conventional/Fixed Scheduling,1669-2405 kg/ha,2405.00000,Value is already in kg/ha.,"Gezira Research Station Farm, Wad Medani, Sudan",NaN,14.401180,NaN,True,...,0.710000,Maximum value of the range 0.45-0.71 kg/m3,True,False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.21608/sjdr.2020.366016074,https://doi.org/10.21608/sjdr.2020.366016074,2025-09-12 23:58:14,pdf,8
311,Film-Mulched Drip Irrigation,Conventional/Fixed Scheduling,"6218.07 kg ha−1 (2018), 5660.59 kg ha−1 (2019)",5660.59000,Yield value for the year 2019.,"Korla, Xinjiang, China",40°53′N,40.883333,Original value was in degrees and minutes form...,False,...,2.600000,(W 0.8),True,False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3390/agronomy15071674,https://doi.org/10.3390/agronomy15071674,2025-09-13 00:18:02,pdf,9
312,Film-Mulched Drip Irrigation,Conventional/Fixed S

In [ ]:
combined_samples = final_df[["irrigation_scheduling_method", "irrigation_scheduling_method_standardized", "plant_yield", "plant_yield_kg_per_hectare", "plant_yield_notes",
                  "location", "latitude", "latitude_decimal_degrees", "latitude_notes", "lat_filled", "longitude", "longitude_decimal_degrees", "longitude_notes", "long_filled",
                  "altitude", "altitude_numeric", "altitude_notes", "altitude_filled",
                  "plant_type", "plant_type_standardized", "plant_type_notes",
                  "water_productivity", "water_productivity_kg_per_m^3", "water_productivity_notes", "water_productivity_valid", "water_productivity_filled",
                  "file_path", "doi", "doi_url", "processed_at", "content source", "relevance_score"]]
combined_samples

,irrigation_scheduling_method,irrigation_scheduling_method_standardized,plant_yield,plant_yield_kg_per_hectare,plant_yield_notes,location,latitude,latitude_decimal_degrees,latitude_notes,lat_filled,...,water_productivity_kg_per_m^3,water_productivity_notes,water_productivity_valid,water_productivity_filled,file_path,doi,doi_url,processed_at,content source,relevance_score
0,Soil moisture sensor-based method (T-VWC),Soil Moisture-Based,66.77 t ha−1,60572.74245,This value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,40.178600,Original value was in degrees with a direction...,False,...,17.370000,NaN,NaN,False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
1,Accumulated radiation combined with soil moist...,Soil Moisture-Based,58.98 t ha−1,53505.77130,Yield value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,40.178600,Original value was in degrees with a direction...,False,...,20.520000,NaN,NaN,False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
2,Crop evapotranspiration estimated method using...,Evapotranspiration-Based,64.54 t ha−1,58549.71990,Value is given in tons per hectare.,Xiaotangshan National Precision Agriculture Ex...,40.1786° N,40.178600,Original value was in degrees with a direction...,False,...,25.070000,NaN,NaN,False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,https://doi.org/10.1038/s41598-024-52972-x,2025-09-12 01:42:16,pdf,9
4,Regulated Deficit Irrigation,Deficit/Partial Irrigation,2.406 kg m-2,24060.00000,"Value is in kilograms per square meter, which ...","Maringá, Paraná, Brazil",23º25′S,-23.416667,Converted from degrees and minutes format (23º...,False,...,13.583452,NaN,NaN,True,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1093/jxb/erv034,https://doi.org/10.1093/jxb/erv034,2025-09-12 04:06:41,pdf,9
5,Regulated Deficit Irrigation,Deficit/Partial Irrigation,2.540 kg m-2,25400.00000,"Value is in kg per square meter, which can be ...","Maringá, Paraná, Brazil",23º25′S,-23.416667,Converted from degrees and minutes (23º25′S) t...,False,...,13.583452,NaN,NaN,True,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1093/jxb/erv034,https://doi.org/10.1093/jxb/erv034,2025-09-12 04:06:41,pdf,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309,"100% ETc, 4-day interval",Evapotranspiration-Based,"136.78 t/ha (Spring 2023), 50.10 t/ha (Autumn ...",45449.96850,Yield value for Autumn 2023.,"Yangling, Shaanxi, China",34°20′N,34.333333,Original value was in degrees and minutes form...,False,...,19.300000,Autumn 2023,True,False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3390/agronomy15051068,https://doi.org/10.3390/agronomy15051068,2025-09-12 17:23:02,pdf,10
310,Timing of last irrigation,Conventional/Fixed Scheduling,1669-2405 kg/ha,2405.00000,Value is already in kg/ha.,"Gezira Research Station Farm, Wad Medani, Sudan",NaN,14.401180,NaN,True,...,0.710000,Maximum value of the range 0.45-0.71 kg/m3,True,False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.21608/sjdr.2020.366016074,https://doi.org/10.21608/sjdr.2020.366016074,2025-09-12 23:58:14,pdf,8
311,Film-Mulched Drip Irrigation,Conventional/Fixed Scheduling,"6218.07 kg ha−1 (2018), 5660.59 kg ha−1 (2019)",5660.59000,Yield value for the year 2019.,"Korla, Xinjiang, China",40°53′N,40.883333,Original value was in degrees and minutes form...,False,...,2.600000,(W 0.8),True,False,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3390/agronomy15071674,https://doi.org/10.3390/agronomy15071674,2025-09-13 00:18:02,pdf,9
312,Film-Mulched Drip Irrigation,Conventional/Fixed S

In [ ]:
info = []
last_df = combined_samples.copy()
for col in ["location", "latitude_decimal_degrees", "longitude_decimal_degrees", "altitude_numeric", "plant_type_standardized", "plant_yield_kg_per_hectare", "water_productivity_kg_per_m^3", "irrigation_scheduling_method"]:
  valid = combined_samples.loc[(combined_samples[col] != "NA") & (combined_samples[col].notna()) & (combined_samples["content source"] == "pdf")]
  total_pdf = combined_samples.loc[last_df["content source"] == "pdf"]
  info.append({
      'feature': col,
      'total_samples': len(total_pdf),
      'valid_samples': len(valid),
      'percentage': float(len(valid)/len(total_pdf)) * 100.0
  })
info = pd.DataFrame(info)
info.sort_values(by= "percentage", ascending= False, inplace= True)
info

,feature,total_samples,valid_samples,percentage
0,location,295,295,100.0
1,latitude_decimal_degrees,295,295,100.0
2,longitude_decimal_degrees,295,295,100.0
3,altitude_numeric,295,295,100.0
4,plant_type_standardized,295,295,100.0
5,plant_yield_kg_per_hectare,295,295,100.0
6,water_productivity_kg_per_m^3,295,295,100.0
7,irrigation_scheduling_method,295,295,100.0


In [ ]:
combined_samples.to_csv("/content/drive/MyDrive/1:1_Sruthi_Manoj/Dataset/csv_files/Extra analysis/cleaned_samples (8).csv")